In [16]:
pip install ndjson

Note: you may need to restart the kernel to use updated packages.


In [17]:
PROJECT_ID = "training1emakia"  # @param {type:"string"}
print(PROJECT_ID)

training1emakia


In [18]:
BUCKET_NAME = "{MY_NAME}_batch_prediction2"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"
print(BUCKET_NAME)
print(BUCKET_URI)

{MY_NAME}_batch_prediction2
gs://{MY_NAME}_batch_prediction2


In [19]:
#create the bucket 
!gsutil mb -b on -l us-central1 gs://{MY_NAME}_batch_prediction2

Creating gs://{MY_NAME}_batch_prediction2/...
ServiceException: 409 A Cloud Storage bucket named '{MY_NAME}_batch_prediction2' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [5]:
#create the bucket 
#!gsutil mb -b on -l us-central1 gs://{MY_NAME}_batch_prediction/output

CommandException: The mb command requires a URL that specifies a bucket.
"gs://{MY_NAME}_batch_prediction2/output" is not valid.


In [20]:
import pandas as pd


gcs_source = 'gs://{MY_NAME}_batch_prediction2/dataforbatchprediction_csv.csv'
data = pd.read_csv(
    gcs_source,      # relative python path to subdirectory
    sep=',',           # Tab-separated value file.
    #quotechar="'",        # single quote allowed as quote character
    usecols=['text']  # Only load the three columns specified.
    #na_values=['.', '??']       # Take any '.' or '??' values as NA
)
print(data.keys())
print(data.text[0])
#print(data[1])

Index(['text'], dtype='object')
@BradleyCullina3 @SenErikaGeiss @GovWhitmer Many agencies do have restrictions on foot and vehicle chases… but there’s a balance between choosing not to chase for risks, and letting everyone avoid legal accountability if they simply flee from officers.


In [21]:
# Instantiate the Storage client and create the new bucket
from google.cloud import  storage

input_file_name = "batch-prediction-input.jsonl"

storage_client = storage.Client()
bucket = storage_client.bucket(BUCKET_NAME)
# Iterate over the prediction instances, creating a new TXT file
# for each.
input_file_data = []
for count, t in enumerate(data.text):
    instance_name = f"input_{count}.txt"
    instance_file_uri = f"{BUCKET_URI}/{instance_name}"
    # Add the data to store in the JSONL input file.
    tmp_data = {"content": instance_file_uri, "mimeType": "text/plain"}
    input_file_data.append(tmp_data)

    # Create the new instance file
    blob = bucket.blob(instance_name)
    blob.upload_from_string(t)

input_str = "\n".join([str(d) for d in input_file_data])
file_blob = bucket.blob(f"{input_file_name}")
file_blob.upload_from_string(input_str)